# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.preprocessing import Normalizer
clf = Normalizer()
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


Normalizer()

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lOutputs;
	}

	tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {

		tTable lTable;

		double lNorm = sqrt(Feature_0 * Feature_0 + Feature_1 * Feature_1 + Feature_2 * Feature_2 + Feature_3 * Feature_3) + 1e-14;

		lTable["Feature_0"] = { Feature_0 / lNorm };
		lTable["Feature_1"] = { Feature_1 / lNorm };
		lTable["Feature_2"] = { Feature_2 / lNorm };
		lTable["Feature_3"] = { Feature_3 / lNorm };

		return lTable;
	}

	tTable compute_model_outputs_from_table( tTable const & iTable) {
		tTable lTable = compute_features(iTable.at("Feature_0")[0], iTable.at("Feature_1")[0], iTable.at("Feature_2")[0], iTable.at("

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_139970810439376.exe /tmp/sklearn2sql_cpp_139970810439376.cpp'
idx,Feature_0,Feature_1,Feature_2,Feature_3
0,0.80377277301538,0.55160876579487,0.22064350631795,0.03152050090256
1,0.82813287338688,0.50702012656339,0.23660939239625,0.03380134177089
2,0.80533307538050,0.54831188111013,0.22275170170099,0.03426949256938
3,0.80003024746206,0.53915081894182,0.26087942852024,0.03478392380270
4,0.79096499646041,0.56949479745150,0.22147019900892,0.03163859985842
5,0.78417498628181,0.56634860120353,0.24686990308872,0.05808703602087
6,0.78010935569635,0.57660256725383,0.23742458651628,0.05087669711063
7,0.80218491851981,0.54548574459347,0.24065547555594,0.03208739674079
8,0.80642365615178,0.53150650064549,0.25658934513920,0.03665562073417
9,0.81803119003537,0.51752993655299,0.25041771123532,0.01669451408235
10,0.80373518807355,0.55070744368003,0.22325977446488,0.02976796992865
11,0.78699100294074,0.55745196041636,0.2623

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,0.677679,0.327155,0.595890,0.280419
74,74,0.767417,0.347736,0.515608,0.155882
9,9,0.818031,0.517530,0.250418,0.016695
88,88,0.729924,0.391031,0.534409,0.169447
25,25,0.826475,0.495885,0.264472,0.033059
5,5,0.784175,0.566349,0.246870,0.058087
48,48,0.798370,0.557353,0.225954,0.030127
117,117,0.692991,0.341996,0.602992,0.197997
83,83,0.707795,0.318508,0.601626,0.188745
105,105,0.709537,0.280080,0.616177,0.196056


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_transform = pd.DataFrame(clf.transform(X), columns=cpp_output.columns[1:]);
skl_output = pd.concat([skl_output_key, skl_output_transform] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,0.677679,0.327155,0.595890,0.280419
74,74,0.767417,0.347736,0.515608,0.155882
9,9,0.818031,0.517530,0.250418,0.016695
88,88,0.729924,0.391031,0.534409,0.169447
25,25,0.826475,0.495885,0.264472,0.033059
5,5,0.784175,0.566349,0.246870,0.058087
48,48,0.798370,0.557353,0.225954,0.030127
117,117,0.692991,0.341996,0.602992,0.197997
83,83,0.707795,0.318508,0.601626,0.188745
105,105,0.709537,0.280080,0.616177,0.196056


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Feature_0_skl,Feature_1_skl,Feature_2_skl,Feature_3_skl,idx_cpp,Feature_0_cpp,Feature_1_cpp,Feature_2_cpp,Feature_3_cpp
114,114,0.677679,0.327155,0.595890,0.280419,114,0.677679,0.327155,0.595890,0.280419
74,74,0.767417,0.347736,0.515608,0.155882,74,0.767417,0.347736,0.515608,0.155882
9,9,0.818031,0.517530,0.250418,0.016695,9,0.818031,0.517530,0.250418,0.016695
88,88,0.729924,0.391031,0.534409,0.169447,88,0.729924,0.391031,0.534409,0.169447
25,25,0.826475,0.495885,0.264472,0.033059,25,0.826475,0.495885,0.264472,0.033059
5,5,0.784175,0.566349,0.246870,0.058087,5,0.784175,0.566349,0.246870,0.058087
48,48,0.798370,0.557353,0.225954,0.030127,48,0.798370,0.557353,0.225954,0.030127
117,117,0.692991,0.341996,0.602992,0.197997,117,0.692991,0.341996,0.602992,0.197997
83,83,0.707795,0.318508,0.601626,0.188745,83,0.707795,0.318508,0.601626,0.188745
105,105,0.709537,0.280080,0.616177,0.196056,105,0.709537,0.280080,0.616177,0.196056


In [14]:
for col in cpp_output.columns:
    lDiff = cpp_skl_join[col + "_skl"] - cpp_skl_join[col + "_cpp"]
    print(lDiff.describe())

count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    1.500000e+02
mean     1.472156e-15
std      2.975997e-15
min     -4.107825e-15
25%     -1.332268e-15
50%      1.998401e-15
75%      4.107825e-15
max      6.106227e-15
dtype: float64
count    1.500000e+02
mean     5.282811e-16
std      2.823499e-15
min     -4.607426e-15
25%     -1.817990e-15
50%      5.551115e-16
75%      2.886580e-15
max      5.884182e-15
dtype: float64
count    1.500000e+02
mean     2.814415e-16
std      2.839961e-15
min     -4.635181e-15
25%     -2.324529e-15
50%      3.330669e-16
75%      2.636780e-15
max      5.662137e-15
dtype: float64
count    1.500000e+02
mean     6.212739e-16
std      2.852168e-15
min     -4.905798e-15
25%     -1.734723e-15
50%      8.604228e-16
75%      3.031429e-15
max      5.273559e-15
dtype: float64
